In [19]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/questions/stack_questions_api.csv
/kaggle/input/use/tensorflow2/default/1/saved_model.pb
/kaggle/input/use/tensorflow2/default/1/variables/variables.index
/kaggle/input/use/tensorflow2/default/1/variables/variables.data-00000-of-00001


In [20]:
import pandas as pd

# Charger le dataset
df = pd.read_csv("/kaggle/input/questions/stack_questions_api.csv")

# Afficher les colonnes et les premières lignes
print(df.columns)
df.head()

Index(['question_id', 'title', 'body', 'tags', 'creation_date', 'score',
       'view_count', 'answer_count', 'is_answered'],
      dtype='object')


,question_id,title,body,tags,creation_date,score,view_count,answer_count,is_answered
0,11227809,Why is processing a sorted array faster than p...,"<p>In this C++ code, sorting the data (<em>bef...","['java', 'c++', 'performance', 'cpu-architectu...",1340805096,27457,1941240,25,True
1,927358,How do I undo the most recent local commits in...,<p>I accidentally committed the wrong files to...,"['git', 'version-control', 'git-commit', 'undo']",1243620554,27061,16139015,103,True
2,2003505,How do I delete a Git branch locally and remot...,<p>Failed Attempts to Delete a Remote Branch:<...,"['git', 'version-control', 'git-branch', 'git-...",1262653935,20365,12869902,41,True
3,292357,What is the difference between &#39;git pull&#...,"<p>What are the differences between <a href=""h...","['git', 'version-control', 'git-pull', 'git-fe...",1226742669,13995,3674790,37,True
4,231767,What does the &quot;yield&quot; keyword do in ...,"<p>What functionality does the <a href=""https:...","['python', 'iterator', 'generator', 'yield']",1224800471,13086,3450087,52,True


In [22]:
import tensorflow_hub as hub

# Charger le modèle localement depuis Kaggle
model = hub.load("/kaggle/input/use/tensorflow2/default/1")

In [23]:
sentences = ["Hello world", "This is a test"]
embeddings = model(sentences)

# Pour voir la taille de l'encodage (512 dimensions normalement)
print(embeddings.shape)

(2, 512)


sentences est une liste de 2 phrases.

Le modèle Universal Sentence Encoder (USE) produit un vecteur d'embedding de dimension 512 pour chaque phrase.

Donc embeddings.shape nous donne (2, 512) = 2 phrases × 512 dimensions.

In [39]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score

X = model(df['title'].tolist())
X = np.array(X)

X_train, X_test, y_train, y_test = train_test_split(X, y_filtered, test_size=0.2, random_state=42)

tags_list = [tags.split() for tags in df['tags']]
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(tags_list)

# Analyse fréquence des labels
freq_labels = y.sum(axis=0)
labels_all = np.where(freq_labels == len(y))[0]      # présents dans TOUS les exemples
labels_rare = np.where(freq_labels < 3)[0]           # moins de 3 occurrences
print(f"Labels présents dans tous les exemples : {[mlb.classes_[i] for i in labels_all]}")
print(f"Labels rares (<3 occurrences) : {[mlb.classes_[i] for i in labels_rare]}")

# Filtrage des labels
keep_labels = np.where((freq_labels < len(y)) & (freq_labels >= 3))[0]
y_filtered = y[:, keep_labels]
filtered_classes = [mlb.classes_[i] for i in keep_labels]

print(f"Nombre de labels avant filtre : {len(mlb.classes_)}")
print(f"Nombre de labels après filtre : {len(filtered_classes)}")

clf = OneVsRestClassifier(LogisticRegression(max_iter=1000))
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

f1_micro = f1_score(y_test, y_pred, average='micro', zero_division=0)
f1_macro = f1_score(y_test, y_pred, average='macro', zero_division=0)

print(f"F1-score micro: {f1_micro:.3f}")
print(f"F1-score macro: {f1_macro:.3f}")


Labels présents dans tous les exemples : []
Labels rares (<3 occurrences) : ["'.net',", "'access-modifiers']", "'actionscript',", "'algorithm',", "'alias']", "'alignment',", "'alignment']", "'android',", "'android-activity',", "'android-edittext',", "'android-emulator',", "'android-input-method',", "'android-layout',", "'android-softkeyboard',", "'angularjs']", "'angularjs-factory',", "'angularjs-provider']", "'angularjs-service',", "'ansi-colors']", "'append',", "'architecture',", "'argument-unpacking']", "'arguments',", "'array-of-dict']", "'arrays']", "'assignment-operator']", "'async-await',", "'asynchronous',", "'asynchronous']", "'authentication',", "'autocomplete']", "'background-color']", "'bash',", "'big-o',", "'bit-manipulation',", "'bitwise-operators']", "'boolean-expression',", "'boolean-operations',", "'branch-prediction']", "'branching-and-merging']", "'browser',", "'browser']", "'c++',", "'c++-faq',", "'c-preprocessor']", "'caret',", "'case-insensitive']", "'centering']"